vamos a aplicar ahora lso conocimientos adquiridos de la guía básica para desarrollar nuestra versión de la api

In [5]:
import json

with open("data/knowledge_base.json", "r", encoding="utf-8") as file:
    database = json.load(file)


In [8]:
type(database)

dict

In [17]:
first_key = next(iter(database))

# Mostrar la clave y el valor asociado
database[first_key[:200]]


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nYour Brain on Music: Tearjerkers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Main Content\n\n\n\n\n\n\n\n\n\n\r\n                        <strong data-cart-timer="" role="text"></strong>\r\n                    \n\n\n\n\n\nNavigation\n\n\n\nKennedy Center\n\n\n\n\n\nKennedy Center\n\n\n\n\nPage Navigation\n\n\nWhat’s On\n\n\nVisit\n\n\nDigital Stage\n\n\nSupport\n\n\nEducation\n\n\nOur Story\n\n\nMemorial\n\n\nShop\n\n\n\n\nPartner Organizations\n\n\nNational Symphony Orchestra\n\n\nWashington National Opera\n\n\n\n\nUtility\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation\n\n\n\n\nKennedy Center\n\n\n\n\n\n\n\nUtility\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle Menu\n\n\n\nPage Navigation\n\n\nWhat’s On\n\n\nVisit\n\n\nDigital Stage\n\n\nSupport\n\n\nEducation\n\n\nOur Story\n\n\nMemorial\n\n\nShop\n\n\nPartner Organizations\n\n\nNational Symphony Orchestra\n\n\nWashington Nati

In [18]:
import re

def preprocess_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    text = re.sub(r'<[^>]+>', '', text)

    text = re.sub(r"[^a-zA-Z0-9\u00C0-\u00FF\s.,!?;:'-]", '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [19]:
clean_database = {}
for key, value in database.items():
    clean_database[key] = preprocess_text(value)

clean_database

{'https://www.kennedy-center.org/education/resources-for-educators/classroom-resources/media-and-interactives/media/music/your-brain-on-music/your-brain-on-music/your-brain-on-music-tearjerkers/': "Your Brain on Music: Tearjerkers Skip to Main Content Navigation Kennedy Center Kennedy Center Page Navigation Whats On Visit Digital Stage Support Education Our Story Memorial Shop Partner Organizations National Symphony Orchestra Washington National Opera Utility Navigation Kennedy Center Utility Toggle Menu Page Navigation Whats On Visit Digital Stage Support Education Our Story Memorial Shop Partner Organizations National Symphony Orchestra Washington National Opera Education Resources for Educators Digital Resources Library Media Interactives Your Brain on Music YBoM: Tearjerkers Music Science Popular Culture Your Brain on Music: Tearjerkers Putting the Sad in Sad Songs Warning: this article contains excerpts from some of the saddest pieces of music ever written. Lesson Content Tearjerk

In [20]:
#  removed the ones for which we were unable to extract information
del clean_database['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3779798/']
del clean_database['https://www.psypost.org/new-research-uncovers-atonal-musics-distinct-emotional-and-neural-effects/']
del clean_database['https://online.ucpress.edu/mp/article/40/3/202/195230/The-Perceptual-and-Emotional-Consequences-of']

In [21]:
print(clean_database.keys())

dict_keys(['https://www.kennedy-center.org/education/resources-for-educators/classroom-resources/media-and-interactives/media/music/your-brain-on-music/your-brain-on-music/your-brain-on-music-tearjerkers/', 'https://dl.acm.org/doi/fullHtml/10.1145/3461615.3485419', 'https://www.unprofesor.com/musica/tipos-de-cadencia-musical-3912.html?utm_source=chatgpt.com#anchor_1', 'https://eldiaadiariomusica.wordpress.com/2013/06/23/sentido-y-personalidad-de-las-tonalidades/'])


In [22]:
with open("data/clean_knowledge_base.json", "w", encoding="utf-8") as file:
    json.dump(clean_database, file, ensure_ascii=False, indent=4)
# lo guardamo

#### Vamos a checkear que tal funciona nuestra api para obtener las features necesarias

In [23]:
import requests
import json
from dotenv import load_dotenv
import os

# loading the craft knowledge database 
with open("data/clean_knowledge_base.json", "r", encoding="utf-8") as file:
    data = json.load(file)

with open('data/music_modes.json', 'r', encoding="utf-8") as f:
    modes = json.load(f)


In [24]:
user_text = """
Today I feel a bit strange, like one of those days when everything is fine but something feels out of place. 
I am calm, but there is this sense that I could be doing more or feeling more. 
It's like being on pause, not sad, but not entirely joyful either. 
I suppose it would be something soft and calm, with a touch of introspection.
"""

clean_user_text = preprocess_text(user_text)
print(clean_user_text)

Today I feel a bit strange, like one of those days when everything is fine but something feels out of place. I am calm, but there is this sense that I could be doing more or feeling more. It's like being on pause, not sad, but not entirely joyful either. I suppose it would be something soft and calm, with a touch of introspection.


In [25]:
prompt_base = (
    "You are an expert in music and a psychologist specializing in the relationship between music and emotions. "
    "Your task is to analyze the text provided by a user, understand the underlying emotional state, and translate it into specific musical characteristics.\n\n"
    "You must return a dict with the following structure, feature_name:  feature_value_given.\n\n"
    "A SPECIFIC VALUE FOR EACH FEATURE MUST BE GIVEN AS A USEFUL VALUE\n\n"
    "If user includes a specific tonality, construct the cadence in orden to satisfy its request\n\n"
    "Extract the following musical features, justifying below the feature dict your choices based on the text:\n\n"
    "1. Tempo: The speed of the music (fast or slow) and how it relates to the described emotional energy.\n"
    "2. Intensity/Dynamics: Volume (crescendos, diminuendos) and its connection to perceived tension or calm.\n"
    "3. Timbre: The quality of sound (dark or bright) and its influence on the emotional atmosphere.\n"
    "4. Rhythm: Rhythmic pattern (regular or irregular) and how it evokes stability or emotional unease.\n"
    "5. Harmonic progression: Provide an exact chord progression (e.g., C-G-Am-F or I, IV, V, I) that reflects the emotional state.\n"
    "6. Melody: Direction and shape (ascending or descending) and how it reflects joy, sadness, or other emotions.\n"
    "7. Tonality/Mode: Key and mode (major, minor, dorian, etc.) and its influence on the emotional color of the text.\n"
    "8. Articulation: Playing style (staccato, legato, etc.) and its relationship to the expressiveness described.\n"
    "9. Silence: Use of pauses or silences between notes to evoke introspection, suspense, or tranquility.\n\n"
    "After the characteristics, develop a sequence of musical notes (C, A, Eb, ...) in the form of a melody in accordance with the tonality of the cadence you have chosen."
    "Additionally, use the provided additional data to enrich your analysis and relate musical characteristics to relevant information. "
    "Combine your skills in music and emotional psychology to provide a detailed and precise response.\n\n"
    "User's text:\n{user_text}\n\n"
    "Additional data:\n{additional_data}, {muical_modes}\n"
)

In [29]:
musical_modes = "\n".join([f"- {key}: {value}" for key, value in modes.items()])
musical_modes[50:150]

"one - Tone - Semitone - Tone - Tone - Tone - Semitone', 'sonority': 'Bright, stable, cheerful.', 'ex"

In [31]:
knw_data = "\n".join([f"- {key}: {value}" for key, value in data.items()])
knw_data[100:200]

'eractives/media/music/your-brain-on-music/your-brain-on-music/your-brain-on-music-tearjerkers/: Your'

In [32]:
combined_data = knw_data + "\n" + musical_modes

In [34]:
final_prompt = prompt_base.format(user_text=clean_user_text, additional_data=combined_data)
final_prompt[1000:1100]

'r emotional unease.\n5. Harmonic progression: Provide an exact chord progression (e.g., C-G-Am-F or I'

In [35]:
payload = {
    "contents": [
        {
            "parts": [{"text": final_prompt}]
        }
    ]
}

load_dotenv("data/api_key1_google.env")
api_key = os.getenv("API_KEY") #set de api key

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_key}"
headers = {"Content-Type": "application/json"}

#snd the request
response = requests.post(url, headers=headers, json=payload)

#proocess the response
if response.status_code == 200:
    result = response.json()
    print(result['candidates'][0]['content']['parts'][0]['text'])
else:
    print(f"Error {response.status_code}: {response.json()}")

The user's text describes a state of calm but with an underlying sense of something being "out of place," a feeling of being "on pause," neither sad nor joyful, but introspective. This suggests a gentle, contemplative mood, not overtly emotional but with a subtle undercurrent of something unresolved.

Here's a musical interpretation reflecting this emotional state:

```json
{
  "Tempo": "Andante (moderately slow)",
  "Intensity/Dynamics": "Piano to mezzo piano, with subtle crescendos and diminuendos to emphasize introspective moments and create a sense of gentle unfolding.",
  "Timbre": "Warm, mellow timbre. Instruments like a solo cello, acoustic guitar, or a muted trumpet would suit this mood. Avoid harsh or bright timbres.",
  "Rhythm": "Predominantly regular, with occasional subtle rhythmic variations to add interest without disrupting the calm. Simple, even note values.",
  "Harmonic progression": "Am - G - C - F (vi - V - I - IV)  This progression in A minor creates a gentle, mel

obtenemos una respuesta bastante contundente parece ser. Nos da una base en la que probar a construir una pequeña melodía

vamos a probar con otros textos de distintos usuarios a ver que dice

In [38]:
user_text_depressed = """
It's one of those days where everything feels heavy, like I'm sinking into myself. 
Even the smallest things seem pointless, and no matter what I do, it feels like there's no real way out. 
There's this emptiness that just sits there, quiet but overwhelming. 
If it were music, it would be slow and haunting, something that pulls you deeper with every note.
"""
clean_depressed_text = preprocess_text(user_text_depressed)

In [39]:
final_prompt_depressed = prompt_base.format(user_text=clean_depressed_text, additional_data=combined_data)

In [40]:
payload = {
    "contents": [
        {
            "parts": [{"text": final_prompt_depressed}]
        }
    ]
}

load_dotenv("data/api_key1_google.env")
api_key = os.getenv("API_KEY") #set de api key

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_key}"
headers = {"Content-Type": "application/json"}

#snd the request
response = requests.post(url, headers=headers, json=payload)

#proocess the response
if response.status_code == 200:
    result = response.json()
    print(result['candidates'][0]['content']['parts'][0]['text'])
else:
    print(f"Error {response.status_code}: {response.json()}")

Based on the user's text and the provided additional data, here's an analysis of the emotional state and its translation into musical characteristics:

**Musical Characteristics:**

```json
{
  "Tempo": "Largo (40-60 bpm)",
  "Intensity/Dynamics": "Piano to pianissimo with occasional, subtle crescendos that quickly decrescendo, mirroring the overwhelming but quiet nature of the emptiness.",
  "Timbre": "Dark, somber. Instruments like cello, viola, or a low-register string quartet would be suitable.  Avoid bright or sharp timbres.",
  "Rhythm": "Irregular, with rubato (flexible tempo) to emphasize the feeling of sinking and unease. Simple, repetitive rhythmic figures might underscore the feeling of being trapped.",
  "Harmonic progression": "Am - F - C - G - Am (vi - IV - i - V - vi) -  This progression utilizes a minor tonality (A minor) which reinforces the feeling of sadness and heaviness. The movement towards the tonic (i) is delayed and ambiguous, reflecting the sense of no way out

In [41]:
user_text_joyful = """
Today feels bright, like the world has this gentle glow to it. 
Everything seems to fit, and even small things bring a kind of quiet happiness. 
If I could describe it as music, it would be in B-flat major, with a light and playful melody, something that lifts you up and keeps you smiling.
"""

clean_joy_text = preprocess_text(user_text_joyful)
clean_joy_text

'Today feels bright, like the world has this gentle glow to it. Everything seems to fit, and even small things bring a kind of quiet happiness. If I could describe it as music, it would be in B-flat major, with a light and playful melody, something that lifts you up and keeps you smiling.'

In [42]:
final_prompt_joy = prompt_base.format(user_text=clean_joy_text, additional_data=combined_data)

payload = {
    "contents": [
        {
            "parts": [{"text": final_prompt_joy}]
        }
    ]
}

load_dotenv("data/api_key1_google.env")
api_key = os.getenv("API_KEY") #set de api key

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_key}"
headers = {"Content-Type": "application/json"}

#snd the request
response = requests.post(url, headers=headers, json=payload)

#proocess the response
if response.status_code == 200: #oood traffic
    result = response.json()
    print(result['candidates'][0]['content']['parts'][0]['text'])
else:
    print(f"Error {response.status_code}: {response.json()}")


The user's text describes a feeling of bright, gentle joy and contentment, even in small things.  The specified tonality of B-flat major strongly supports this emotional description.  Major keys are generally associated with happiness and optimism in Western musical traditions. The description of a "light and playful melody" further reinforces the positive emotional state.

Here's a breakdown of the musical characteristics based on the text and additional data:


```json
{
  "Tempo": "Andante (moderately slow)",
  "Intensity/Dynamics": "mf (moderately loud) with occasional pianissimo (very soft) passages for emphasis and contrast.",
  "Timbre": "Bright, clear timbre.  Suitable instruments might include flute, glockenspiel, strings (violins, cellos), perhaps a light piano",
  "Rhythm": "Mostly regular, but with some subtle rhythmic variation to avoid monotony.  A predominantly triple meter (e.g., 3/4) could create a sense of gentle flow and grace",
  "Harmonic progression": "I - IV - V 

ya hemos demostrado que funciona más o menos como espérabamos, con un claro margen de depuración y mejora.

vamos a modificar un pequeño detalle para que tú como usuario seas quien lo pruebe

In [43]:
# try it!!!

user_text = input('Please enter your text about how you are feeling currently : ')

while not user_text.strip():
    user_text = input("Text cannot be empty. Please enter your text: ")

final_prompt= prompt_base.format(user_text=user_text, additional_data=combined_data)


In [44]:
print(user_text)

im feelling a little bit odd, it seems that it doesnt matter how much effort i apply to studing i will never manage to live up to the expectations people have aboutme. Im tires y just want to relax and stop thinking


In [ ]:
payload = {
    "contents": [
        {
            "parts": [{"text": final_prompt}]
        }
    ]
}

load_dotenv("data/api_key1_google.env") #this is our api_key (must be private and unshared), get yours in https://aistudio.google.com/app/apikey
api_key = os.getenv("API_KEY") #set de api key

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={api_key}"
headers = {"Content-Type": "application/json"}

#snd the request
response = requests.post(url, headers=headers, json=payload)

#proocess the response
if response.status_code == 200: #oood traffic
    result = response.json()
    print(result['candidates'][0]['content']['parts'][0]['text'])
else:
    print(f"Error {response.status_code}: {response.json()}")

Based on the user's text expressing feelings of being "a little bit odd," overwhelmed by expectations, tired, and desiring relaxation, the underlying emotional state is a blend of sadness, weariness, and a longing for peace.  The additional data reinforces the association between specific musical characteristics and emotions.

Here's a musical interpretation translated from the emotional state:

```json
{
  "Tempo": "Andante (moderately slow)",
  "Intensity/Dynamics": "Piano to mezzo-piano, with occasional diminuendos to pianissimo during moments of introspection and then a gradual crescendo towards the end suggesting a hopeful resolution.",
  "Timbre": "Dark, warm, mellow.  Instruments like cello, viola, and muted trumpet would suit the mood.",
  "Rhythm": "Irregular, with some syncopation to reflect the feeling of unease and instability, but punctuated by moments of simple, regular beats for brief respite.",
  "Harmonic progression": "Am - G - C - F (vi - V - I - IV) in A minor. This